In [1]:
import numpy as np
import sympy as sm
import chart_studio.plotly as py
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default='notebook'

In [11]:
def eval_exp(exp, point):
    return float(exp.evalf(subs={k:v for k,v in zip(exp.free_symbols, point)}))

def grad(exp):
    return [exp.diff(s) for s in exp.free_symbols]

def grad_func(exp):
    gradf = grad(exp)
    return lambda vec: np.array([eval_exp(g, vec) for g in gradf])

def gradient_descent(exp, cpoint, learning_rate=0.1, eps=1e-6, epochs=1000):
    grad_f = grad_func(exp)
    path = []
    epoch = 0

    while np.linalg.norm(grad_f(cpoint)) > eps and epoch < epochs:
        cpoint = cpoint - learning_rate * grad_f(cpoint)
        func_value = eval_exp(exp, cpoint)
        path.append(np.r_[cpoint, func_value])
        epoch += 1

    return np.array(path)


In [27]:
def get_bounds(pts: np.array, delta=2):
    x = pts.T[0].min()-delta, pts.T[0].max()+delta
    y = pts.T[1].min()-delta, pts.T[1].max()+delta
    return x, y

def build_surface(exp, x_b=(-10, 10), y_b=(-10, 10), grid=(100, 100)):
    x = np.linspace(*x_b, grid[0])
    y = np.linspace(*y_b, grid[1])
    
    # !!! [Y x X]
    z = np.array([[eval_exp(exp, [i, j]) for i in x] for j in y])
    
    return x, y, z

In [58]:
def visualize_path(exp, grad_path):
    x, y, z = build_surface(exp, *get_bounds(grad_path))
    
    surface = go.Surface(
        x=x, y=y, z=z,
        colorscale='Turbo',
        contours_z=dict(
            show=True,
            usecolormap=True,
            highlightcolor="limegreen",
            project_z=True
        )
    )
    
    line = go.Scatter3d(
        x=grad_path.T[0], y=grad_path.T[1], z=grad_path.T[2],
        mode='markers',
        marker=dict(
            size=6,
            color=grad_path.T[2],
            colorscale='gray'
        )
    )
    
    layout = go.Layout(
        title='[3D] Gradient descent on func y='+str(exp),
#         width=1000,
#         height=1000,
        autosize=True
    )
    
    fig = go.Figure(data=[surface, line], layout=layout)

    fig.show()

In [59]:
def visualize_contours(exp, grad_path):
    x, y, z = build_surface(exp, *get_bounds(grad_path))
    gx, gy, gz = grad_path.T
    
    contour = go.Contour(
        x=x, y=y, z=z,
        colorscale='Turbo'
    )

    line = go.Scatter(
        x=gx, y=gy,
        mode='markers',
        marker=dict(
            color=gz,
            size=8,
            colorscale='gray'
        )
    )
    
    layout = go.Layout(
        title='[Contour] Gradient descent on func y='+str(exp),
        autosize=True
    )
    fig = go.Figure(data=[contour, line], layout=layout)
    
    fig.show()

### Ackley function
$$f(x, y) = -e + 20e^{-\sqrt{\frac{x^2 + y^2}{50}}} + e^{\frac{1}{2}(\cos{2\pi x} + \cos{2\pi y})}$$

In [70]:
def ackley_func():
    x, y = sm.symbols('x, y')
    return (
        - sm.exp(1)
        + 20 * sm.exp(-sm.sqrt((x**2 + y**2)/50))
        + sm.exp(0.5 * (sm.cos(2*sm.pi*x) + sm.cos(2*sm.pi*y)))
    )

exp = -ackley_func()
path = gradient_descent(exp, [5, 5])

In [71]:
visualize_path(exp, path)
visualize_contours(exp, path)